In [ ]:
!bash /content/drive/MyDrive/single-person-pose-estimation/dataset/get_data.sh

Getting train2017
--2022-03-18 02:12:44--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.65.48
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.65.48|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip         6%[>                   ]   1.21G  45.1MB/s    eta 6m 29s 

In [ ]:
import time
from datetime import datetime, timedelta
import sys

In [ ]:
sys.path.append('/content/drive/MyDrive/single-person-pose-estimation')

In [ ]:
from configs import default_config as cfg
from coco_df import gen_trainval_df
from gen_tfrecords import gen_TFRecords

In [ ]:
train_df, valid_df = gen_trainval_df(cfg)

In [ ]:
start = time.time()
gen_TFRecords(train_df, cfg, is_train = True)
total_time = time.time() - start
print("Total time: {}".format(str(timedelta(seconds=total_time))))

In [ ]:
start = time.time()
gen_TFRecords(valid_df, cfg, is_train = False)
total_time = time.time() - start
print("Total time: {}".format(str(timedelta(seconds=total_time))))

In [ ]:
#check size, good size per file ~100Mb
!ls -lh "dataset/tfrecords/train" 
!ls -lh "dataset/tfrecords/valid" 



---



---

Just testing

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
### For reading
def parse_tfrecord_fn(example):
    feature_description = {
        "ann_id": tf.io.FixedLenFeature([], tf.int64),
        "image_id": tf.io.FixedLenFeature([], tf.int64),
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_path": tf.io.FixedLenFeature([], tf.string),
        "width": tf.io.FixedLenFeature([], tf.int64),
        "height": tf.io.FixedLenFeature([], tf.int64),
        "keypoints/x": tf.io.VarLenFeature(tf.float32),
        "keypoints/y": tf.io.VarLenFeature(tf.float32),
        "keypoints/vis": tf.io.VarLenFeature(tf.int64),
        "keypoints/num": tf.io.FixedLenFeature([], tf.int64)
    }
    
    example = tf.io.parse_single_example(example, feature_description)
    example["image"] = tf.image.decode_image(example["image"], channels = 3, dtype = tf.float32, expand_animations =False)
    example["keypoints/x"] = tf.sparse.to_dense(example["keypoints/x"])
    example["keypoints/y"] = tf.sparse.to_dense(example["keypoints/y"])
    example["keypoints/vis"] = tf.sparse.to_dense(example["keypoints/vis"])
    return example
  ### example
def gen_examples_from_tfrecord(filepath, example_num):
  raw_dataset = tf.data.TFRecordDataset(filepath)
  parsed_dataset = raw_dataset.map(parse_tfrecord_fn)
  for example in parsed_dataset.take(example_num):
      for key in example.keys():
          if key != "image":
              print(f"{key}: {example[key]}")

      xcoords = example["keypoints/x"].numpy()
      ycoords = example["keypoints/y"].numpy()
      image = example["image"]
      h, w, c = image.shape
      print(f"Image shape: {image.shape}")
      plt.figure(figsize=(7, 7))
  
      plt.imshow(image)
      plt.scatter(xcoords  , ycoords  , marker = "o") # for heatmap size
      plt.show()  

In [ ]:
gen_examples_from_tfrecord('dataset/tfrecords/valid/file_valid_00-2048.tfrec', 10)



---



---

IF run on local machine skip the following

In [ ]:
!zip -r 'dataset/tfrecords.zip' 'dataset/tfrecords'

In [ ]:
!cp 'dataset/tfrecords.zip' 'drive/MyDrive/single-person-pose-estimation/dataset'